In [27]:
# load and clean data

import pandas as pd

# Load the two Excel files
matcher = pd.read_excel('Compustat_Ticker_Name_Match_202511.xlsx')
master = pd.read_excel('Compustat_CIK_GVKEY_Master_NonFinancial_All.xlsx')

# to title casing

matcher['Firmname_adj'] = matcher['Company'].str.title()
master['Firmname_adj'] = master['company'].str.title()

master.to_excel('Compustat_CIK_GVKEY_Master_NonFinancial_All.xlsx', index=False)
matcher.to_excel('Compustat_Ticker_Name_Match_202511.xlsx', index=False)




In [ ]:
# punctuation, extra spaces, and common suffixes such as “Inc.”, “Corp.”, “Co.”, “Ltd.”, and “Corporation”

import re
import string

def clean_company_name(text):
    if pd.isna(text):
        return text
    text = str(text).strip()

    suffixes = [
        r'\b(inc\.?|incorporated)\b',
        r'\b(corp\.?|corporation)\b', 
        r'\b(co\.?|company)\b',
        r'\b(ltd\.?|limited)\b',
        r'\b(llc)\b',
        r'\b(lp)\b',
        r'\b(llp)\b',
        r'\b(pllc)\b',
        r'\b(pa)\b',
        r'\b(pc)\b',
        r'\b(dba)\b',
        r'\b(and associates)\b',
        r'\b(& associates)\b',
        r'\b(associates)\b'
    ]
    
    for suffix in suffixes:
        text = re.sub(suffix, '', text, flags=re.IGNORECASE)
    
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

matcher['Firmname_adj'] = matcher['Firmname_adj'].apply(clean_company_name)
master['Firmname_adj'] = master['Firmname_adj'].apply(clean_company_name)

master.to_excel('Compustat_CIK_GVKEY_Master_NonFinancial_All.xlsx', index=False)
matcher.to_excel('Compustat_Ticker_Name_Match_202511.xlsx', index=False)



In [24]:
# Matching the tickers of dataset 1 in dataset 2

matcher['matched_ticker'] = matcher['ticker'].isin(master['ticker']).astype(int)
matcher.to_excel('Compustat_Ticker_Name_Match_202511.xlsx', index=False)

In [ ]:
# fuzzy name checking using fuzzywuzzy

from fuzzywuzzy import fuzz, process

def fuzzy_match_companies(company_name, company_list):
    """
    Fuzzy match company name and return match category and best match
    """
    if pd.isna(company_name):
        return 0, None, 0
    match = process.extractOne(company_name, company_list, scorer=fuzz.ratio)
    
    if match:
        score = match[1]
        matched_name = match[0]
        
        if score >= 90:
            return 1, matched_name, score
        elif score >= 80:
            return 2, matched_name, score
        else:
            return 0, None, score
    else:
        return 0, None, 0
    

company_names_master = master['Firmname_adj'].dropna().tolist()
results = matcher['Firmname_adj'].apply(lambda x: fuzzy_match_companies(x, company_names_master))
matcher['Matched_firmname'] = [r[0] for r in results]
matcher['Firmname_adj_Compustat'] = [r[1] for r in results]
matcher['Similarity_Score'] = [r[2] for r in results]

matcher.to_excel('Compustat_Ticker_Name_Match_202511.xlsx', index=False)




In [ ]:
# get the original name for Firname_Compustat
master_unique = master.drop_duplicates(subset=['Firmname_adj'], keep='first')

matcher['Firmname_Compustat'] = matcher['Firmname_adj_Compustat'].map(
    master_unique.set_index('Firmname_adj')['company']
)

matcher.to_excel('Compustat_Ticker_Name_Match_202511.xlsx', index=False)

In [33]:
# build the Matched_confirm column

matcher['Matched_confirm'] = matcher.apply(
    lambda row: 1 if (row['Matched_ticker'] == 1 and row['Matched_firmname'] in [1, 2]) else 0, axis=1
)
matcher.to_excel('Compustat_Ticker_Name_Match_202511.xlsx', index=False)